## Endpoint A/B Testing Live Evaluation tests

In [ ]:
%%time
%matplotlib inline
from datetime import datetime, timedelta
import time
import os
import boto3
import re
import json
from sagemaker import get_execution_role, session
from sagemaker.s3 import S3Downloader, S3Uploader

region = boto3.Session().region_name
role = get_execution_role()
sm_session = session.Session(boto3.Session())
sm = boto3.Session().client("sagemaker")
sm_runtime = boto3.Session().client("sagemaker-runtime")

# You can use a different bucket, but make sure the role you chose for this notebook
# has the s3:PutObject permissions. This is the bucket into which the model artifacts will be uploaded
bucket = sm_session.default_bucket()
prefix = "sagemaker/DEMO-VariantTargeting"

### Run Inference test against endpoint

In [ ]:
endpoint_name = 'DEMO-xgb-churn-pred-2022-07-23-03-13-03'

with open("test_data/test_sample_tail_input_cols.csv", "r") as f:
    buffer = f.read()

buflist = buffer.split('\n')
    
for row in buflist:
    print(".", end="", flush=True)
    payload = row.rstrip("\n")
    if payload:
        sm_runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType="text/csv", Body=payload)
    time.sleep(0.5)

## Test to show endpoint switching based on weights

In [ ]:
test = buflist[3]

In [ ]:
for x in range(0,10):
    op = sm_runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType="text/csv", Body=test)
    print(op['Body'].read().decode())
    

## Test to show inference performance while updating endpoints 

In [ ]:
import time

while True:
    start = time.time()
    for x in range(0,10):
        start = time.time()
        op = sm_runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType="text/csv", Body=test)
    end = time.time()
    print(end - start, end='\r', flush=True)
    time.sleep(5)